# Part I. ETL Pipeline for Pre-Processing the Files

For more information on partition keys, composite keys, clustering keys and primary keys please see the [stack overflow article](https://stackoverflow.com/questions/24949676/difference-between-partition-key-composite-key-and-clustering-key-in-cassandra#:~:text=A%20partition%20key%20is%20the,the%20ordering%20within%20a%20partition).

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/Users/tyler/t-lanigan/data-engineer-nanodegree/project1b-data-modeling-with-cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1

The goal of the first query is to get the `artist`, `song_title` and `song_length` given a `session_id` and and `item_in_session`. The `session_id` as the partition key and the `item_in_session` as the clustering key can be combined to make a composite primary key.

In [8]:
query = """
    CREATE TABLE IF NOT EXISTS session_items
    (session_id int, item_in_session int, artist text, song_title text, song_length float,
    PRIMARY KEY(session_id, item_in_session))
    """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_items (session_id, item_in_session, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        artist, first_name, gender, item_in_session, last_name, length, level, location, session_id, song_title, user_id = line
        session.execute(query, (int(session_id), int(item_in_session), artist, song_title, float(length)))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
result = session.execute("""SELECT artist, song_title, song_length
                            FROM session_items WHERE session_id = 338
                            AND item_in_session = 4""")
result.all()

[Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)]

### Query 2

The second query's goal s are to get the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.

We need to include the `artist`, `song`, and `firstName` and `lastName`, as well as the `userid` and `sessionid` and `itemInSession`. The primary key can be the `userId` as the paritition key with the `sessionId` and the `itemInSession` as clustering keys to make it unique.

In [11]:
query = """
    CREATE TABLE IF NOT EXISTS user_items
    (user_id int, session_id int, item_in_session int, artist text, song_title text, first_name text, last_name text,
    PRIMARY KEY(user_id, session_id, item_in_session))
    """

try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_items (user_id, session_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        artist, first_name, gender, item_in_session, last_name, length, level, location, session_id, song_title, user_id = line
        session.execute(query, (int(user_id), int(session_id), int(item_in_session), artist, song_title, first_name, last_name))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
result = session.execute("""SELECT item_in_session, artist, song_title, first_name, last_name
                            FROM user_items WHERE user_id = 10
                            AND session_id = 182""")
for row in result.all():
    print(row)

Row(item_in_session=0, artist='Down To The Bone', song_title="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(item_in_session=1, artist='Three Drives', song_title='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(item_in_session=2, artist='Sebastien Tellier', song_title='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(item_in_session=3, artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


### Query 3
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own

We need to keep track of the song_title, first name, last name and userid. The `song_title` as the partition key and the user_id as the clustering key can be combined to make a primary key. It is ok in this case that the user might listen to a song more than once and thus have more than one entry in the data that would get deleted by using this primary key, however this suites our purpose, becuase we don't need diplicates for this query.



In [14]:
query = """
    CREATE TABLE IF NOT EXISTS music_history
    (song_title text, first_name text, last_name text, user_id int,
    PRIMARY KEY(song_title, user_id))"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_history (song_title, first_name, last_name, user_id)"
        query = query + " VALUES (%s, %s, %s, %s)"
        artist, first_name, gender, item_in_session, last_name, length, level, location, session_id, song_title, user_id = line
        session.execute(query, (song_title, first_name, last_name, int(user_id)))

#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
result = session.execute("""SELECT song_title, first_name, last_name
                            FROM music_history
                            WHERE song_title = 'All Hands Against His Own'""")
result.all()

[Row(song_title='All Hands Against His Own', first_name='Jacqueline', last_name='Lynch'),
 Row(song_title='All Hands Against His Own', first_name='Tegan', last_name='Levine'),
 Row(song_title='All Hands Against His Own', first_name='Sara', last_name='Johnson')]

### Drop the tables before closing out the sessions

In [17]:
session.execute("""DROP TABLE user_items""")

session.execute("""DROP TABLE session_items""")

session.execute("""DROP TABLE music_history""")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()